# Title

## How to Use:

0. **Run the first code cell** to load all imports and utilities (optimized for performance)
1. **Change the stock**: Edit `user_input = 'AAPL'` to any stock symbol
2. **Run the selection cell** to load the stock data
3. **Run the dashboard cell** to create the analysis

## Dashboard Components:

- **Main Chart**: Stock price with moving averages and Bollinger Bands
- **Volume**: Trading volume in millions (color-coded by price change)
- **RSI**: Relative Strength Index with overbought/oversold levels
- **Volatility**: Annualized volatility percentage
- **Cumulative Returns**: Total returns from start
- **MACD**: Moving Average Convergence Divergence with signal line and histogram
- **Max Drawdown**: Peak-to-trough decline percentage over time
- **Stochastic Oscillator**: %K and %D lines with overbought/oversold levels
- **Returns Distribution**: Histogram of daily returns
- **On-Balance Volume**: Cumulative volume indicator
- **Rolling Sharpe Ratio**: Risk-adjusted performance over 252-day windows

## Examples:
- `user_input = 'GOOGL'` for Google
- `user_input = 'TSLA'` for Tesla
- `user_input = 'NVDA'` for NVIDIA
- `user_input = 'MSFT'` for Microsoft

# Table of Contents

- [Setup and Configuration](#setup-and-configuration)
- [Stock Selection](#stock-selection)  
- [Technical Analysis Dashboard](#technical-analysis-dashboard)
- [Cointegration Analysis](#cointegration-analysis)
- [Company Information](#company-information)
- [Analyst Price Targets](#analyst-price-targets)
- [Market Intelligence](#market-intelligence)

[Back to Top](#table-of-contents)

## Setup and Configuration

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import scipy.stats as stats
from typing import Dict, List, Optional
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

# Additional imports for display and web requests
from IPython.display import Markdown, display
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
import json

# Utility functions
def get_ticker(symbol: str):
    """Get or create a yfinance Ticker object."""
    if 'ticker' in globals() and hasattr(ticker, 'ticker') and ticker.ticker == symbol:
        return ticker
    return yf.Ticker(symbol)

def fetch_stock_data(symbol: str, period: str = '1y'):
    """Fetch stock data, using cache if available."""
    cache_key = f"{symbol}_{period}"
    if cache_key in stocks:
        return stocks[cache_key]
    ticker = get_ticker(symbol)
    data = ticker.history(period=period)
    if not data.empty:
        stocks[cache_key] = data
    return data

def setup_date_formatting(ax, period):
    """Set up appropriate date formatting for x-axis based on period."""
    from matplotlib.dates import DateFormatter, MonthLocator, YearLocator, DayLocator, WeekdayLocator
    
    if period in ['1d', '5d']:
        # For very short periods, show hours/minutes
        ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))
        ax.xaxis.set_major_locator(DayLocator())
    elif period in ['1wk', '1mo']:
        # For weeks/months, show days
        ax.xaxis.set_major_formatter(DateFormatter('%b %d'))
        ax.xaxis.set_major_locator(DayLocator(interval=7))  # Weekly ticks
    elif period in ['3mo', '6mo']:
        # For 3-6 months, show weeks/months
        ax.xaxis.set_major_formatter(DateFormatter('%b %d'))
        ax.xaxis.set_major_locator(MonthLocator(interval=1))
    elif period in ['1y', '2y']:
        # For 1-2 years, show months
        ax.xaxis.set_major_formatter(DateFormatter('%b %Y'))
        ax.xaxis.set_major_locator(MonthLocator(interval=3))  # Quarterly ticks
    elif period in ['5y', '10y', 'max']:
        # For longer periods, show years
        ax.xaxis.set_major_formatter(DateFormatter('%Y'))
        ax.xaxis.set_major_locator(YearLocator())
    else:
        # Default fallback
        ax.xaxis.set_major_formatter(DateFormatter('%b %Y'))

# Module-level state for storing stock data
stocks: Dict[str, pd.DataFrame] = {}
stock_metrics: Dict[str, pd.DataFrame] = {}
stock_symbols: List[str] = []
selected_stock: Optional[str] = None

# Additional Utility Functions

In [ ]:
def make_cnn_api_request(api_url: str, timeout: int = 10) -> Optional[requests.Response]:
    """Make a request to CNN Business API with retry logic and multiple headers."""
    headers_list = [
        {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Accept": "application/json, text/javascript, */*; q=0.01",
            "Referer": "https://www.cnn.com/",
        },
        {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
            "Accept": "application/json, text/javascript, */*; q=0.01",
            "Referer": "https://www.cnn.com/",
        },
        {"User-Agent": "curl/7.88.1", "Accept": "*/*"},
    ]

    session = requests.Session()
    retries = Retry(total=2, backoff_factor=0.8, status_forcelist=[429, 500, 502, 503, 504])
    session.mount("https://", HTTPAdapter(max_retries=retries))

    resp = None
    for h in headers_list:
        try:
            resp = session.get(api_url, headers=h, timeout=timeout)
            if resp.status_code == 200:
                break
            time.sleep(0.5)
        except requests.exceptions.RequestException:
            resp = None
            time.sleep(0.5)

    return resp

def display_structured_data_as_markdown(data, title: str, symbol: str, field_mappings: Dict[str, str] = None):
    """Display structured data (list of dicts) as formatted markdown."""
    md_lines = []

    try:
        # Normalize pandas structures to list/dict
        if isinstance(data, (pd.DataFrame, pd.Series)):
            data = data.to_dict(orient='records') if isinstance(data, pd.DataFrame) else data.tolist()

        # If a single item was returned as a dict, wrap it
        if isinstance(data, dict):
            data = [data]

        # Header
        if not data:
            md_lines.append(f"**{title}:** Not available for {symbol}")
        else:
            md_lines.append(f"### {title} for {symbol}")
            md_lines.append('')

            # Display items
            for i, item in enumerate(data, 1):
                try:
                    if isinstance(item, dict):
                        # Use field mappings if provided, otherwise try common fields
                        if field_mappings:
                            primary = item.get(field_mappings.get('primary', 'name'), 'Unknown')
                            secondary = item.get(field_mappings.get('secondary', 'title'), 'Unknown')
                        else:
                            # Default mappings
                            primary = (item.get('name') or item.get('fullName') or item.get('personName') or
                                     item.get('person_name') or item.get('symbol') or 'Unknown')
                            secondary = (item.get('title') or item.get('position') or item.get('role') or
                                       item.get('value') or item.get('shares') or 'Unknown')

                        details = f"**{secondary}** — {primary}"

                        # Add extra fields if present
                        extras = []
                        extra_fields = ['since', 'startDate', 'appointed', 'age', 'years', 'pctHeld', 'value']
                        for field in extra_fields:
                            if field in item and item[field] is not None:
                                if field in ['pctHeld']:
                                    extras.append(f"{item[field]:.2f}%")
                                elif field in ['value']:
                                    extras.append(f"${item[field]:,.0f}")
                                else:
                                    extras.append(f"{field.replace('pctHeld', '% held').replace('startDate', 'since')}: {item[field]}")

                        if extras:
                            details += " (" + ", ".join(extras) + ")"

                        md_lines.append(f"{i}. {details}")
                    else:
                        # Fallback for other types
                        md_lines.append(f"{i}. {item}")
                except Exception:
                    md_lines.append(f"{i}. Unknown item")

        display(Markdown("\n\n".join(md_lines)))
    except Exception as e:
        display(Markdown(f"**Error displaying {title}:** {e}"))

def calculate_technical_indicators(data: pd.DataFrame) -> pd.DataFrame:
    """Calculate common technical indicators for stock data."""
    df = data.copy()

    # Moving averages
    df['MA_20'] = df['Close'].rolling(window=20).mean()
    df['MA_50'] = df['Close'].rolling(window=50).mean()

    # RSI
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # MACD
    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']

    # Bollinger Bands
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['STD_20'] = df['Close'].rolling(window=20).std()
    df['Upper_Band'] = df['SMA_20'] + (df['STD_20'] * 2)
    df['Lower_Band'] = df['SMA_20'] - (df['STD_20'] * 2)

    # Stochastic Oscillator
    high_14 = df['High'].rolling(window=14).max()
    low_14 = df['Low'].rolling(window=14).min()
    df['%K'] = 100 * ((df['Close'] - low_14) / (high_14 - low_14))
    df['%D'] = df['%K'].rolling(window=3).mean()

    # On-Balance Volume
    df['OBV'] = (df['Volume'] * ((df['Close'] > df['Close'].shift(1)).astype(int) * 2 - 1)).cumsum()

    return df

[Back to Top](#table-of-contents)

## Code

Note:
cmd+shift+h = collapse all input

## Stock Selection

In [ ]:
user_input = 'AAPL'  # Change this to any stock symbol (e.g., 'AAPL', 'GOOGL', 'MSFT', etc.)
period = '1y'  # Change this to desired period (e.g., '1d', '1wk', '6mo', '1y', '2y', 'max' etc.) - x-axis labels will auto-adjust

In [ ]:
# STOCK SELECTION
# Change the stock symbol below to any stock you want to analyze

selected_stock = user_input.upper()
print(f"Fetching data for {selected_stock}...")

# Fetch stock data
data = fetch_stock_data(selected_stock, period)

if not data.empty:
    stock_symbols.append(selected_stock)
    
    # Calculate technical indicators using utility function
    data = calculate_technical_indicators(data)
    
    # Calculate additional metrics
    data['Daily_Return'] = data['Close'].pct_change()
    data['Volatility'] = data['Daily_Return'].rolling(window=20).std() * np.sqrt(252)
    data['Cumulative_Return'] = (1 + data['Daily_Return']).cumprod() - 1
    
    stock_metrics[selected_stock] = data
    ticker = get_ticker(selected_stock)  # Ensure ticker is set
    print(f"Successfully loaded {selected_stock}")
else:
    print(f"No data found for {selected_stock}")

## Simple Stock Selection

**How to change stocks:**
1. In the cell above, change `user_input = 'AAPL'` to any stock symbol you want
2. Examples: `'GOOGL'`, `'MSFT'`, `'TSLA'`, `'NVDA'`, `'AMZN'`, etc.
3. Run the cell above, then run the dashboard cell below

**That's it! Super simple.**

## Technical Analysis Dashboard

In [ ]:
# Create dashboard visualization
print(f"Creating dashboard for {selected_stock}...")

# Period mapping for readable titles
period_names = {
    '1d': '1 Day',
    '5d': '5 Days', 
    '1mo': '1 Month',
    '3mo': '3 Months',
    '6mo': '6 Months',
    '1y': '1 Year',
    '2y': '2 Years',
    '5y': '5 Years',
    '10y': '10 Years',
    'ytd': 'Year to Date',
    'max': 'Maximum'
}

# Get readable period name
period_display = period_names.get(period, period)

data = stock_metrics[selected_stock]

# Additional calculations for new charts
# Bollinger Bands
upper_band = data['MA_20'] + (data['Close'].rolling(20).std() * 2)
lower_band = data['MA_20'] - (data['Close'].rolling(20).std() * 2)

# MACD
ema_12 = data['Close'].ewm(span=12).mean()
ema_26 = data['Close'].ewm(span=26).mean()
macd = ema_12 - ema_26
signal = macd.ewm(span=9).mean()
histogram = macd - signal

# Maximum Drawdown
cumulative = (1 + data['Daily_Return']).cumprod()
running_max = cumulative.cummax()
drawdown = (cumulative - running_max) / running_max * 100

# Stochastic Oscillator
low_14 = data['Low'].rolling(14).min()
high_14 = data['High'].rolling(14).max()
k_percent = 100 * ((data['Close'] - low_14) / (high_14 - low_14))
d_percent = k_percent.rolling(3).mean()

# On-Balance Volume
obv = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()

# Rolling Sharpe Ratio
rolling_sharpe = (data['Daily_Return'].rolling(60).mean() / data['Daily_Return'].rolling(60).std()) * np.sqrt(252)

# Create figure with subplots
fig = plt.figure(figsize=(14, 16))
gs = GridSpec(6, 3, height_ratios=[4, 1, 1.5, 1.5, 1.5, 2], hspace=0.4, wspace=0.3)

# Main price chart with moving averages and Bollinger Bands
ax_main = fig.add_subplot(gs[0, :])
ax_main.plot(data.index, data['Close'], color="#114f7a", label=f'{selected_stock} Close', linewidth=2.5)
ax_main.plot(data.index, data['MA_20'], color="#114e7ae0", alpha=0.8, linestyle='--', label='20-day MA', linewidth=1.5)
ax_main.plot(data.index, data['MA_50'], color="#114e7ac8", alpha=0.8, linestyle=':', label='50-day MA', linewidth=1.5)
ax_main.fill_between(data.index, lower_band, upper_band, color='gray', alpha=0.2, label='Bollinger Bands')
ax_main.set_title(f'{selected_stock} Stock Price with Moving Averages & Bollinger Bands ({period_display})', fontsize=16, fontweight='bold', pad=20)
ax_main.set_ylabel('Price ($)', fontsize=12)
ax_main.legend(fontsize=10, loc='upper left')
ax_main.grid(True, alpha=0.3)
ax_main.tick_params(labelbottom=False)

# Add current price annotation
current_price = data['Close'].iloc[-1]
ax_main.annotate(f'${current_price:.2f}', xy=(data.index[-1], current_price), xytext=(10, 10), 
                textcoords='offset points', bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7), 
                fontsize=10, fontweight='bold')

# Volume chart
ax_volume = fig.add_subplot(gs[1, :], sharex=ax_main)
volume_data = data['Volume'] / 1e6
bars = ax_volume.bar(data.index, volume_data, color='#d62728', alpha=0.6, width=1)
ax_volume.set_ylabel('Volume\n(Millions)', fontsize=10)
ax_volume.grid(True, alpha=0.3)
ax_volume.tick_params(labelsize=9)

# Color bars based on price change
for i, bar in enumerate(bars):
    if i > 0:
        if data['Close'].iloc[i] > data['Close'].iloc[i-1]:
            bar.set_color('#00ff00')
        else:
            bar.set_color('#ff0000')

# RSI chart
ax_rsi = fig.add_subplot(gs[2, 0])
ax_rsi.plot(data.index, data['RSI'], color='#114f7a', linewidth=2, alpha=0.8)
ax_rsi.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
ax_rsi.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
ax_rsi.set_ylim(0, 100)
ax_rsi.set_title('RSI', fontweight='bold', fontsize=11)
ax_rsi.set_ylabel('RSI', fontsize=9)
ax_rsi.legend(fontsize=7)
ax_rsi.tick_params(axis='both', labelsize=8)

# Add latest RSI annotation
if not data['RSI'].empty and not np.isnan(data['RSI'].iloc[-1]):
    latest_rsi = data['RSI'].iloc[-1]
    ax_rsi.annotate(f'{latest_rsi:.1f}', xy=(data.index[-1], latest_rsi), xytext=(5, 5), 
                   textcoords='offset points', fontsize=8, fontweight='bold', 
                   bbox=dict(boxstyle='round,pad=0.2', facecolor='#9467bd', alpha=0.2))

# Volatility chart
ax_vol = fig.add_subplot(gs[2, 1])
volatility_pct = data['Volatility'] * 100
ax_vol.plot(data.index, volatility_pct, color='#114f7a', linewidth=2, alpha=0.8)
ax_vol.set_title('Volatility', fontweight='bold', fontsize=11)
ax_vol.set_ylabel('Volatility (%)', fontsize=9)
ax_vol.tick_params(axis='both', labelsize=8)

# Add latest volatility annotation
if not volatility_pct.empty and not np.isnan(volatility_pct.iloc[-1]):
    latest_vol = volatility_pct.iloc[-1]
    ax_vol.annotate(f'{latest_vol:.1f}', xy=(data.index[-1], latest_vol), xytext=(5, 5), 
                   textcoords='offset points', fontsize=8, fontweight='bold', 
                   bbox=dict(boxstyle='round,pad=0.2', facecolor='#114f7a', alpha=0.2))

# Cumulative Returns chart
ax_cum = fig.add_subplot(gs[2, 2])
cumulative_returns_pct = data['Cumulative_Return'] * 100
ax_cum.plot(data.index, cumulative_returns_pct, color='#114f7a', linewidth=2, alpha=0.8)
ax_cum.set_title('Cumulative Returns', fontweight='bold', fontsize=11)
ax_cum.set_ylabel('Cumulative Returns (%)', fontsize=9)
ax_cum.tick_params(axis='both', labelsize=8)

# Add latest cumulative return annotation
if not cumulative_returns_pct.empty and not np.isnan(cumulative_returns_pct.iloc[-1]):
    latest_cum = cumulative_returns_pct.iloc[-1]
    ax_cum.annotate(f'{latest_cum:.1f}', xy=(data.index[-1], latest_cum), xytext=(5, 5), 
                   textcoords='offset points', fontsize=8, fontweight='bold', 
                   bbox=dict(boxstyle='round,pad=0.2', facecolor='#114f7a', alpha=0.2))

# MACD chart
ax_macd = fig.add_subplot(gs[3, :2])
ax_macd.plot(data.index, macd, label='MACD', color='blue', linewidth=1.5)
ax_macd.plot(data.index, signal, label='Signal', color='red', linewidth=1)
ax_macd.bar(data.index, histogram, color='gray', alpha=0.5, width=1)
ax_macd.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax_macd.set_title('MACD', fontweight='bold', fontsize=11)
ax_macd.legend(fontsize=8)
ax_macd.tick_params(axis='both', labelsize=8)

# Maximum Drawdown chart
ax_dd = fig.add_subplot(gs[3, 2])
ax_dd.fill_between(data.index, drawdown, 0, color='red', alpha=0.3)
ax_dd.set_title('Max Drawdown (%)', fontweight='bold', fontsize=11)
ax_dd.set_ylabel('Drawdown (%)', fontsize=9)
ax_dd.tick_params(axis='both', labelsize=8)

# Add latest drawdown annotation
if not drawdown.empty and not np.isnan(drawdown.iloc[-1]):
    latest_dd = drawdown.iloc[-1]
    ax_dd.annotate(f'{latest_dd:.1f}', xy=(data.index[-1], latest_dd), xytext=(5, 5), 
                  textcoords='offset points', fontsize=8, fontweight='bold', 
                  bbox=dict(boxstyle='round,pad=0.2', facecolor='red', alpha=0.2))

# Stochastic Oscillator
ax_stoch = fig.add_subplot(gs[4, 0])
ax_stoch.plot(data.index, k_percent, label='%K', color='blue')
ax_stoch.plot(data.index, d_percent, label='%D', color='red')
ax_stoch.axhline(y=80, color='red', linestyle='--', alpha=0.7, label='Overbought')
ax_stoch.axhline(y=20, color='green', linestyle='--', alpha=0.7, label='Oversold')
ax_stoch.set_title('Stochastic Oscillator', fontweight='bold', fontsize=11)
ax_stoch.legend(fontsize=7)
ax_stoch.tick_params(axis='both', labelsize=8)

# Returns Distribution Histogram
ax_hist = fig.add_subplot(gs[4, 1])
ax_hist.hist(data['Daily_Return'].dropna() * 100, bins=50, alpha=0.7, color='purple', edgecolor='black')
ax_hist.set_title('Daily Returns Distribution', fontweight='bold', fontsize=11)
ax_hist.set_xlabel('Return (%)')
ax_hist.axvline(x=0, color='red', linestyle='--')
ax_hist.tick_params(axis='both', labelsize=8)

# On-Balance Volume
ax_obv = fig.add_subplot(gs[4, 2])
ax_obv.plot(data.index, obv / 1e6, color='orange', linewidth=2)
ax_obv.set_title('On-Balance Volume (Millions)', fontweight='bold', fontsize=11)
ax_obv.grid(True, alpha=0.3)
ax_obv.tick_params(axis='both', labelsize=8)

# Rolling Sharpe Ratio
ax_sharpe_roll = fig.add_subplot(gs[5, :])
ax_sharpe_roll.plot(data.index, rolling_sharpe, color='green', linewidth=2)
ax_sharpe_roll.set_title('Rolling Sharpe Ratio (60-day)', fontweight='bold', fontsize=10)
ax_sharpe_roll.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax_sharpe_roll.grid(True, alpha=0.3)
ax_sharpe_roll.tick_params(axis='both', labelsize=8)

# Set up proper date formatting for x-axis based on period
setup_date_formatting(ax_sharpe_roll, period)

plt.tight_layout()
plt.subplots_adjust(bottom=0.15)
plt.show()

In [ ]:
# Display key metrics with visual cards and mini charts
from IPython.display import HTML, display
from io import BytesIO
import base64

def safe_float(value):
    """Safely convert to float, return None if fails."""
    try:
        return float(value)
    except (TypeError, ValueError):
        return None

def humanize_number(x):
    """Convert large numbers to human-readable format."""
    try:
        x = float(x)
    except Exception:
        return "N/A"
    if pd.isna(x):
        return "N/A"
    abs_x = abs(x)
    if abs_x >= 1e12:
        return f"{x/1e12:.2f}T"
    if abs_x >= 1e9:
        return f"{x/1e9:.2f}B"
    if abs_x >= 1e6:
        return f"{x/1e6:.2f}M"
    if abs_x >= 1e3:
        return f"{x/1e3:.2f}K"
    return f"{x:.0f}"

def get_current_price_and_date(recent_hist, data, info):
    """Get current price and last date from available sources."""
    current_price = None
    last_date = None
    
    if isinstance(recent_hist, pd.DataFrame) and not recent_hist.empty:
        current_price = safe_float(recent_hist['Close'].iloc[-1])
        last_date = recent_hist.index[-1]
    
    if current_price is None and isinstance(data, pd.DataFrame) and not data.empty:
        current_price = safe_float(data['Close'].iloc[-1])
        last_date = data.index[-1]
    
    if current_price is None:
        current_price = info.get('currentPrice') or info.get('regularMarketPrice')
    
    return current_price, last_date

def get_historical_values(data, column='Close', periods=[30, 90, 180, 365]):
    """Get historical values for specified periods (in days) for any column."""
    values = []
    if isinstance(data, pd.DataFrame) and not data.empty and column in data.columns and len(data) > 1:
        # Clean the data first - remove NaN values
        clean_data = data[column].dropna()
        if clean_data.empty:
            return values
            
        for period in periods:
            if len(clean_data) >= period:
                val = clean_data.iloc[-period]
                values.append(safe_float(val))
            elif len(clean_data) > 0:
                # If we don't have enough data, use the oldest available value
                val = clean_data.iloc[0]
                values.append(safe_float(val))
            else:
                values.append(None)
    return values

def create_mini_chart(values, current_value, color='#3b82f6'):
    """Create a mini column chart showing 4 bars."""
    if not values or all(v is None for v in values):
        return None
    
    # Get last 4 values
    values = values[-4:]
    
    # Make sure we have 4 values (pad if needed)
    while len(values) < 4:
        values.insert(0, None)
    
    fig, ax = plt.subplots(figsize=(1.2, 0.7))
    fig.patch.set_alpha(0)
    ax.set_facecolor('none')
    
    # Create 4 bars - all using the same color (blue or specified color)
    x = list(range(4))
    bar_values = [values[i] if i < len(values) and values[i] else 0 for i in range(4)]
    
    # Create bars with rounded corners
    bars = ax.bar(x, bar_values, color=color, width=0.7)
    
    # Add rounded corners to bars
    for bar in bars:
        bar.set_capstyle('round')
        bar.set_joinstyle('round')
    
    # Remove axes
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    plt.tight_layout(pad=0)
    
    # Convert to base64
    buf = BytesIO()
    plt.savefig(buf, format='png', dpi=50, bbox_inches='tight', transparent=True)
    plt.close(fig)
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    
    return f'data:image/png;base64,{img_base64}'

def create_metric_card(name, value, change_pct=None, chart_img=None):
    """Create HTML for a metric card."""
    # Determine color based on change
    if change_pct is not None:
        color = '#10b981' if change_pct > 0 else '#ef4444' if change_pct < 0 else '#6b7280'
        change_symbol = '▲' if change_pct > 0 else '▼' if change_pct < 0 else '●'
        change_html = f'<div style="color: {color}; font-size: 12px; margin-top: 4px;">{change_symbol} {abs(change_pct):.1f}%</div>'
    else:
        change_html = ''
    
    # If there's a chart, use flexbox layout with chart on the right
    if chart_img:
        return f'''
        <div style="
            background: white;
            border: 1px solid #e5e7eb;
            border-radius: 8px;
            padding: 16px;
            min-width: 180px;
            box-shadow: 0 1px 3px rgba(0,0,0,0.1);
            display: flex;
            justify-content: space-between;
            align-items: center;
            gap: 12px;
        ">
            <div style="flex: 1;">
                <div style="color: #6b7280; font-size: 13px; font-weight: 500; margin-bottom: 8px;">{name}</div>
                <div style="color: #111827; font-size: 24px; font-weight: 700;">{value}</div>
                {change_html}
            </div>
            <div style="flex-shrink: 0;">
                <img src="{chart_img}" style="width: 70px; height: 40px;">
            </div>
        </div>
        '''
    else:
        return f'''
        <div style="
            background: white;
            border: 1px solid #e5e7eb;
            border-radius: 8px;
            padding: 16px;
            min-width: 180px;
            box-shadow: 0 1px 3px rgba(0,0,0,0.1);
        ">
            <div style="color: #6b7280; font-size: 13px; font-weight: 500; margin-bottom: 8px;">{name}</div>
            <div style="color: #111827; font-size: 24px; font-weight: 700;">{value}</div>
            {change_html}
        </div>
        '''

def create_performance_card(name, change_pct):
    """Create HTML for a performance card with colored value as main display."""
    if change_pct is None:
        return f'''
        <div style="
            background: white;
            border: 1px solid #e5e7eb;
            border-radius: 8px;
            padding: 16px;
            min-width: 180px;
            box-shadow: 0 1px 3px rgba(0,0,0,0.1);
        ">
            <div style="color: #6b7280; font-size: 13px; font-weight: 500; margin-bottom: 8px;">{name}</div>
            <div style="color: #111827; font-size: 24px; font-weight: 700;">N/A</div>
        </div>
        '''
    
    # Determine color based on change
    color = '#10b981' if change_pct > 0 else '#ef4444' if change_pct < 0 else '#6b7280'
    change_symbol = '▲' if change_pct > 0 else '▼' if change_pct < 0 else '●'
    
    # Determine if this is a percentage or ratio metric
    is_percentage = '%' in name or 'Return' in name or 'Volatility' in name or 'Drawdown' in name or 'VaR' in name
    
    if is_percentage:
        formatted_value = f'{change_symbol} {abs(change_pct):.1f}%'
    else:
        # For ratios like Sharpe, Sortino, Calmar
        formatted_value = f'{change_symbol} {abs(change_pct):.2f}'
    
    return f'''
    <div style="
        background: white;
        border: 1px solid #e5e7eb;
        border-radius: 8px;
        padding: 16px;
        min-width: 180px;
        box-shadow: 0 1px 3px rgba(0,0,0,0.1);
    ">
        <div style="color: #6b7280; font-size: 13px; font-weight: 500; margin-bottom: 8px;">{name}</div>
        <div style="color: {color}; font-size: 24px; font-weight: 700;">{formatted_value}</div>
    </div>
    '''

# Main logic
if 'selected_stock' not in globals() or not selected_stock:
    display(HTML('<div style="color: #dc2626; font-weight: 600;">No stock selected. Set user_input and run the STOCK SELECTION cell first.</div>'))
else:
    # Create ticker
    try:
        ticker = yf.Ticker(selected_stock)
    except Exception as e:
        display(HTML(f'<div style="color: #dc2626;">Error creating ticker for {selected_stock}: {e}</div>'))
        raise
    
    # Fetch data
    recent_hist = ticker.history(period='5d') if ticker else None
    data = stock_metrics.get(selected_stock) if 'stock_metrics' in globals() and isinstance(stock_metrics, dict) else None
    if data is None or (isinstance(data, pd.DataFrame) and data.empty):
        data = ticker.history(period='max') if ticker else None  # Get max history for yearly data
    info = getattr(ticker, 'info', {}) or {}
    
    # Get historical values for charts - YEARLY: 3 years ago, 2 years ago, 1 year ago, current
    # Approximate trading days: 252 per year
    hist_values_price = get_historical_values(data, 'Close', [756, 504, 252, 1])  # 3y, 2y, 1y, now
    hist_values_rsi = get_historical_values(data, 'RSI', [756, 504, 252, 1])
    hist_values_volume = get_historical_values(data, 'Volume', [756, 504, 252, 1])
    
    # Extract current price
    current_price, last_date = get_current_price_and_date(recent_hist, data, info)
    
    # Calculate changes
    price_change_1d = None
    if isinstance(recent_hist, pd.DataFrame) and len(recent_hist) >= 2:
        p0 = safe_float(recent_hist['Close'].iloc[-2])
        p1 = safe_float(recent_hist['Close'].iloc[-1])
        if p0:
            price_change_1d = (p1 - p0) / p0 * 100.0
    
    # Create mini charts for key metrics
    price_chart = None
    if hist_values_price and current_price:
        price_chart = create_mini_chart(hist_values_price, current_price)
    
    # Get additional metrics
    market_cap = info.get('marketCap')
    pe_ratio = info.get('trailingPE')
    eps = info.get('trailingEps')
    pb_ratio = info.get('priceToBook')
    dividend_yield = info.get('dividendYield')
    week_52_high = info.get('fiftyTwoWeekHigh')
    week_52_low = info.get('fiftyTwoWeekLow')
    volume = info.get('volume')
    avg_volume = info.get('averageVolume')
    beta = info.get('beta')
    
    # RSI
    rsi = None
    rsi_chart = None
    if isinstance(data, pd.DataFrame) and 'RSI' in data.columns:
        rsi_series = data['RSI'].dropna()
        if not rsi_series.empty:
            rsi = safe_float(rsi_series.iloc[-1])
            if hist_values_rsi and rsi:
                rsi_chart = create_mini_chart(hist_values_rsi, rsi, '#3b82f6')
    
    # MACD
    macd_value = None
    if isinstance(data, pd.DataFrame) and 'MACD' in data.columns:
        macd_series = data['MACD'].dropna()
        if not macd_series.empty:
            macd_value = safe_float(macd_series.iloc[-1])
    
    # Bollinger %B
    bollinger_position = None
    if isinstance(data, pd.DataFrame) and all(col in data.columns for col in ['Close', 'SMA_20', 'Upper_BB', 'Lower_BB']):
        close_val = safe_float(data['Close'].iloc[-1])
        upper = safe_float(data['Upper_BB'].iloc[-1])
        lower = safe_float(data['Lower_BB'].iloc[-1])
        if close_val and upper and lower and upper != lower:
            bollinger_position = (close_val - lower) / (upper - lower)
    
    # Stochastic %K
    stochastic_k = None
    if isinstance(data, pd.DataFrame) and 'Stochastic_K' in data.columns:
        stoch_series = data['Stochastic_K'].dropna()
        if not stoch_series.empty:
            stochastic_k = safe_float(stoch_series.iloc[-1])
    
    # Volume
    current_volume = None
    volume_chart = None
    if isinstance(data, pd.DataFrame) and 'Volume' in data.columns:
        vol_series = data['Volume'].dropna()
        if not vol_series.empty:
            current_volume = safe_float(vol_series.iloc[-1])
            if hist_values_volume and current_volume:
                volume_chart = create_mini_chart(hist_values_volume, current_volume, '#3b82f6')
    
    # Company info
    company_name = info.get('longName') or info.get('shortName') or selected_stock
    sector = info.get('sector', 'N/A')
    industry = info.get('industry', 'N/A')
    
    # Calculate performance metrics
    perf_30d = None
    perf_90d = None
    perf_6m = None
    perf_1y = None
    
    if isinstance(data, pd.DataFrame) and not data.empty and 'Close' in data.columns:
        if len(data) >= 30:
            perf_30d = ((data['Close'].iloc[-1] - data['Close'].iloc[-30]) / data['Close'].iloc[-30]) * 100
        if len(data) >= 90:
            perf_90d = ((data['Close'].iloc[-1] - data['Close'].iloc[-90]) / data['Close'].iloc[-90]) * 100
        if len(data) >= 126:  # ~6 months
            perf_6m = ((data['Close'].iloc[-1] - data['Close'].iloc[-126]) / data['Close'].iloc[-126]) * 100
        if len(data) >= 252:
            perf_1y = ((data['Close'].iloc[-1] - data['Close'].iloc[-252]) / data['Close'].iloc[-252]) * 100
        elif len(data) > 126:  # If we have more than 6 months but less than 1 year, calculate from oldest available
            perf_1y = ((data['Close'].iloc[-1] - data['Close'].iloc[0]) / data['Close'].iloc[0]) * 100
    
    # Calculate risk metrics
    sharpe_ratio = None
    sortino_ratio = None
    calmar_ratio = None
    annualized_volatility = None
    annualized_return = None
    max_drawdown = None
    var_95 = None
    
    # Require at least 30 days of data instead of 252
    if isinstance(data, pd.DataFrame) and not data.empty and 'Close' in data.columns and len(data) >= 30:
        # Get daily returns
        returns = data['Close'].pct_change().dropna()
        
        if not returns.empty and len(returns) > 0:
            # Annualized Return
            total_return = (data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1
            years = len(data) / 252
            if years > 0:
                annualized_return = ((1 + total_return) ** (1 / years) - 1) * 100
            
            # Annualized Volatility
            annualized_volatility = returns.std() * np.sqrt(252) * 100
            
            # Sharpe Ratio (assuming 0% risk-free rate)
            if annualized_volatility and annualized_volatility > 0:
                sharpe_ratio = (annualized_return or 0) / annualized_volatility
            
            # Max Drawdown
            cumulative = (1 + returns).cumprod()
            running_max = cumulative.expanding().max()
            drawdown = (cumulative - running_max) / running_max
            max_drawdown = drawdown.min() * 100
            
            # Sortino Ratio (downside deviation)
            downside_returns = returns[returns < 0]
            if len(downside_returns) > 0:
                downside_std = downside_returns.std() * np.sqrt(252)
                if downside_std > 0:
                    sortino_ratio = (annualized_return or 0) / (downside_std * 100)
            
            # Calmar Ratio
            if max_drawdown and max_drawdown < 0 and annualized_return:
                calmar_ratio = annualized_return / abs(max_drawdown)
            
            # VaR (95%)
            var_95 = abs(returns.quantile(0.05)) * 100
    
    # Build HTML
    html_parts = []
    
    # Header
    html_parts.append(f'''
    <div style="font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif; padding: 20px;">
        <h2 style="color: #111827; margin-bottom: 8px;">{company_name} ({selected_stock})</h2>
        <div style="color: #6b7280; font-size: 14px; margin-bottom: 24px;">
            {sector} • {industry}
        </div>
    ''')
    
    # Main metrics grid
    html_parts.append('<div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 16px; margin-bottom: 24px;">')
    
    # Price card
    if current_price:
        html_parts.append(create_metric_card(
            'Current Price',
            f'${current_price:.2f}',
            price_change_1d,
            price_chart
        ))
    
    # Market Cap card
    if market_cap:
        html_parts.append(create_metric_card('Market Cap', f'${humanize_number(market_cap)}'))
    
    # P/E Ratio card
    if pe_ratio:
        html_parts.append(create_metric_card('P/E Ratio', f'{pe_ratio:.2f}'))
    
    # EPS card
    if eps:
        html_parts.append(create_metric_card('EPS', f'${eps:.2f}'))
    
    # Dividend Yield card
    if dividend_yield:
        html_parts.append(create_metric_card('Dividend Yield', f'{dividend_yield:.2f}%'))
    else:
        html_parts.append(create_metric_card('Dividend Yield', 'N/A'))
    
    # P/B Ratio card
    if pb_ratio:
        html_parts.append(create_metric_card('P/B Ratio', f'{pb_ratio:.2f}'))
    
    # 52W Range
    if week_52_high and week_52_low:
        html_parts.append(create_metric_card('52W High', f'${week_52_high:.2f}'))
        html_parts.append(create_metric_card('52W Low', f'${week_52_low:.2f}'))
    
    # Volume
    volume_display = volume or current_volume
    if volume_display:
        html_parts.append(create_metric_card('Volume', humanize_number(volume_display), None, volume_chart))
    
    # RSI card
    if rsi:
        html_parts.append(create_metric_card('RSI', f'{rsi:.1f}', None, rsi_chart))
    
    # MACD card
    if macd_value is not None:
        html_parts.append(create_metric_card('MACD', f'{macd_value:.3f}'))
    
    # Bollinger %B card
    if bollinger_position is not None:
        html_parts.append(create_metric_card('Bollinger %B', f'{bollinger_position:.2f}'))
    
    # Stochastic %K card
    if stochastic_k is not None:
        html_parts.append(create_metric_card('Stochastic %K', f'{stochastic_k:.1f}'))
    
    # Beta card
    if beta:
        html_parts.append(create_metric_card('Beta', f'{beta:.2f} (vs S&P 500)'))
    
    html_parts.append('</div>')  # Close main grid
    
    # Risk Metrics section
    html_parts.append('<h3 style="color: #111827; margin-top: 24px; margin-bottom: 16px;">Risk Metrics</h3>')
    html_parts.append('<div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 16px;">')
    
    html_parts.append(create_performance_card('Sharpe Ratio', sharpe_ratio))
    html_parts.append(create_performance_card('Sortino Ratio', sortino_ratio))
    html_parts.append(create_performance_card('Calmar Ratio', calmar_ratio))
    html_parts.append(create_performance_card('Annualized Volatility', annualized_volatility))
    html_parts.append(create_performance_card('Annualized Return', annualized_return))
    html_parts.append(create_performance_card('Max Drawdown', max_drawdown))
    html_parts.append(create_performance_card('VaR (95%)', var_95))
    
    html_parts.append('</div>')  # Close risk metrics grid
    
    # Performance section
    html_parts.append('<h3 style="color: #111827; margin-top: 24px; margin-bottom: 16px;">Performance</h3>')
    html_parts.append('<div style="display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 16px;">')
    
    html_parts.append(create_performance_card('30-Day Return', perf_30d))
    html_parts.append(create_performance_card('90-Day Return', perf_90d))
    html_parts.append(create_performance_card('6-Month Return', perf_6m))
    html_parts.append(create_performance_card('1-Year Return', perf_1y))
    
    html_parts.append('</div>')  # Close performance grid
    html_parts.append('</div>')  # Close main container
    
    # Display the HTML
    display(HTML(''.join(html_parts)))


[Back to Top](#table-of-contents)

# Cointegration Analysis

**Cointegration** tests whether two or more time series share a long-run equilibrium relationship, even though they may drift apart in the short term. This is particularly useful for:

- **Pairs Trading**: Finding stocks that move together
- **Portfolio Construction**: Identifying stocks with stable relationships
- **Risk Management**: Understanding long-term dependencies


## Quick Cointegration Guide

### What is Cointegration?
- **Definition**: A statistical relationship where two or more price series move together in the long run
- **Purpose**: Find stocks that have stable long-term relationships despite short-term divergences
- **Applications**: Pairs trading, portfolio construction, risk management

### How to Interpret Results:

#### Engle-Granger Test (Pairs):
- **P-value < 0.05**: Strong evidence of cointegration 
- **P-value > 0.05**: No evidence of cointegration 

#### Johansen Test (Multiple stocks):
- **Cointegration rank > 0**: Found long-term relationships 
- **Cointegration rank = 0**: No relationships found 

### Trading Strategies:
1. **Pairs Trading**: When spread deviates from mean, trade the divergence
2. **Portfolio Rebalancing**: Use cointegrated stocks for stable portfolios
3. **Risk Management**: Avoid highly correlated positions without cointegration

### Pro Tips:
- Test different time periods (1y, 2y, 5y)
- Consider sector relationships (tech stocks, bank stocks, etc.)
- Monitor changing relationships over time

## How to Use:
1. Set your stock pairs in the cell below
2. Run the cointegration analysis
3. Interpret the results (p-value < 0.05 suggests cointegration)

In [ ]:
# Cointegration Analysis Setup
# Change these stock pairs to test different combinations
stock_pair_1 = 'KO'  # First stock
stock_pair_2 = 'PEP'   # Second stock

# You can also test multiple stocks at once
multi_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']  # Portfolio of stocks to test

print(f"Setting up cointegration analysis for:")
print(f"Pair analysis: {stock_pair_1} vs {stock_pair_2}")
print(f"Multi-stock analysis: {multi_stocks}")

In [ ]:
# Pairwise cointegration analysis
print("PAIRWISE COINTEGRATION ANALYSIS")
print("="*60)

# Fetch data for both stocks
pair_symbols = [stock_pair_1, stock_pair_2]
cointegration_data = {}

for symbol in pair_symbols:
    try:
        data_temp = fetch_stock_data(symbol, "1y")
        if not data_temp.empty:
            cointegration_data[symbol] = data_temp['Close']
        else:
            print(f"Warning: No data found for {symbol}")
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

# Perform Engle-Granger test if we have both stocks
if len(cointegration_data) >= 2:
    stock1_prices = cointegration_data[stock_pair_1]
    stock2_prices = cointegration_data[stock_pair_2]
    
    # Align the data
    aligned_data = pd.concat([stock1_prices, stock2_prices], axis=1).dropna()
    
    if aligned_data.shape[0] >= 30:
        series1 = aligned_data.iloc[:, 0]
        series2 = aligned_data.iloc[:, 1]
        
        # Perform cointegration test
        score, p_value, critical_values = coint(series1, series2)
        
        # Determine conclusion
        if p_value < 0.01:
            conclusion = "STRONG evidence of cointegration (99% confidence)"
            trading_signal = "EXCELLENT for pairs trading"
        elif p_value < 0.05:
            conclusion = "GOOD evidence of cointegration (95% confidence)"
            trading_signal = "GOOD for pairs trading"
        elif p_value < 0.10:
            conclusion = "WEAK evidence of cointegration (90% confidence)"
            trading_signal = "RISKY for pairs trading"
        else:
            conclusion = 'NO evidence of cointegration'
            trading_signal = 'NOT suitable for pairs trading'
        
        print(f"\nCointegration Test Results:")
        print(f"Test Score: {score:.4f}")
        print(f"P-value: {p_value:.4f}")
        print(f"Conclusion: {conclusion}")
        print(f"Trading Signal: {trading_signal}")
        
        if p_value < 0.05:
            print("These stocks ARE cointegrated!")
        else:
            print("These stocks are NOT cointegrated")
            
        # Plot the analysis
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f'Cointegration Analysis: {stock_pair_1} vs {stock_pair_2}', fontsize=16, fontweight='bold')
        
        # Plot 1: Raw prices
        axes[0, 0].plot(series1.index, series1, label=stock_pair_1, linewidth=2)
        axes[0, 0].plot(series2.index, series2, label=stock_pair_2, linewidth=2)
        axes[0, 0].set_title('Price Series')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Plot 2: Normalized prices
        norm_series1 = (series1 / series1.iloc[0]) * 100
        norm_series2 = (series2 / series2.iloc[0]) * 100
        axes[0, 1].plot(norm_series1.index, norm_series1, label=f'{stock_pair_1} (Normalized)', linewidth=2)
        axes[0, 1].plot(norm_series2.index, norm_series2, label=f'{stock_pair_2} (Normalized)', linewidth=2)
        axes[0, 1].set_title('Normalized Price Series (Base = 100)')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Plot 3: Scatter plot
        axes[1, 0].scatter(series1, series2, alpha=0.6, s=20)
        slope, intercept, r_value, p_val_reg, std_err = stats.linregress(series1, series2)
        line = slope * series1 + intercept
        axes[1, 0].plot(series1, line, 'r', linewidth=2, label=f'R² = {r_value**2:.3f}')
        axes[1, 0].set_xlabel(f'{stock_pair_1} Price ($)')
        axes[1, 0].set_ylabel(f'{stock_pair_2} Price ($)')
        axes[1, 0].set_title('Price Relationship Scatter Plot')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Plot 4: Spread
        spread = series2 - (slope * series1 + intercept)
        axes[1, 1].plot(spread.index, spread, color='purple', linewidth=1)
        axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
        axes[1, 1].axhline(y=spread.std(), color='red', linestyle='--', alpha=0.7, label='+1 Std Dev')
        axes[1, 1].axhline(y=-spread.std(), color='red', linestyle='--', alpha=0.7, label='-1 Std Dev')
        axes[1, 1].set_title('Spread (Residuals) - Trading Signal')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        print("Error: Not enough data points for reliable cointegration test")
else:
    print("❌ Could not fetch sufficient data for both stocks")

## Multy-Stock Cointegration

In [ ]:
# Multi-stock cointegration analysis
print("Multi-stock cointegration analysis")
print("="*60)

# Fetch data for multiple stocks
multi_data = {}
for symbol in multi_stocks:
    try:
        data_temp = fetch_stock_data(symbol, "1y")
        if not data_temp.empty:
            multi_data[symbol] = data_temp['Close']
        else:
            print(f"Warning: No data found for {symbol}")
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

if len(multi_data) >= 2:
    # Convert to list for easier handling
    price_series = [multi_data[symbol] for symbol in multi_stocks if symbol in multi_data]
    available_stocks = [symbol for symbol in multi_stocks if symbol in multi_data]
    
    print(f"Analyzing {len(available_stocks)} stocks: {', '.join(available_stocks)}")
    
    # Prepare data for Johansen test
    aligned_data = pd.concat(price_series, axis=1).dropna()
    
    if aligned_data.shape[0] >= 50:
        # Perform Johansen test
        johansen_result = coint_johansen(aligned_data, 0, 1)
        trace_stats = johansen_result.lr1
        critical_values_trace = johansen_result.cvt
        
        cointegration_rank = 0
        for i in range(len(trace_stats)):
            if trace_stats[i] > critical_values_trace[i, 1]:
                cointegration_rank = i + 1
                break
        
        print(f"\nJohansen Test Results:")
        print(f"Cointegration Rank: {cointegration_rank}")
        
        # Plot multi-stock analysis
        normalized_data = aligned_data.div(aligned_data.iloc[0]) * 100
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Plot 1: Raw prices
        for i, col in enumerate(aligned_data.columns):
            axes[0, 0].plot(aligned_data.index, aligned_data[col], label=available_stocks[i], linewidth=2)
        axes[0, 0].set_title('Raw Price Series')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Plot 2: Normalized prices
        for i, col in enumerate(normalized_data.columns):
            axes[0, 1].plot(normalized_data.index, normalized_data[col], label=f'{available_stocks[i]} (Norm)', linewidth=2)
        axes[0, 1].set_title('Normalized Price Series (Base = 100)')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Plot 3: Correlation matrix
        correlation_matrix = aligned_data.corr()
        im = axes[1, 0].imshow(correlation_matrix, cmap='RdYlBu', vmin=-1, vmax=1)
        axes[1, 0].set_xticks(range(len(available_stocks)))
        axes[1, 0].set_yticks(range(len(available_stocks)))
        axes[1, 0].set_xticklabels(available_stocks, rotation=45)
        axes[1, 0].set_yticklabels(available_stocks)
        axes[1, 0].set_title('Correlation Matrix')
        for i in range(len(available_stocks)):
            for j in range(len(available_stocks)):
                axes[1, 0].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}', 
                              ha="center", va="center", color="black", fontweight='bold')
        
        # Plot 4: Rolling correlation
        if len(available_stocks) >= 2:
            rolling_corr = aligned_data.iloc[:, 0].rolling(30).corr(aligned_data.iloc[:, 1])
            axes[1, 1].plot(rolling_corr.index, rolling_corr, linewidth=2, color='purple')
            axes[1, 1].set_title(f'30-Day Rolling Correlation\n{available_stocks[0]} vs {available_stocks[1]}')
            axes[1, 1].axhline(y=0.8, color='green', linestyle='--', alpha=0.7, label='High Correlation')
            axes[1, 1].axhline(y=0.5, color='orange', linestyle='--', alpha=0.7, label='Medium Correlation')
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Additional insights
        if cointegration_rank > 0:
            print(f"\nPORTFOLIO INSIGHTS:")
            print(f"{'='*40}")
            print(f"Cointegration rank: {cointegration_rank}")
            print(f"Trading opportunities:")
            print(f"   • Monitor for temporary divergences from long-run relationship")
            print(f"   • Consider mean-reversion strategies")
            print(f"   • Use portfolio approach rather than individual stock picks")
    else:
        print("Error: Not enough data points for reliable Johansen test")
else:
    print("Could not fetch sufficient data for multi-stock analysis")

[Back to Top](#table-of-contents)

# Under construction - Future Enhancement

This section is planned for additional analysis features. Currently contains basic yfinance info display.

In [ ]:
# Print full yfinance info for the selected stock (lightweight)
pprint(ticker.info)

In [ ]:
# Display company officers using utility function
try:
    # Prefer single_metrics if available, otherwise fall back to ticker.info
    officers = None
    if 'single_metrics' in locals() and single_metrics:
        officers = single_metrics.get('companyOfficers')

    if officers is None:
        if 'ticker' not in globals():
            ticker = yf.Ticker(selected_stock)
        info = getattr(ticker, 'info', {}) or {}
        officers = info.get('companyOfficers')

    display_structured_data_as_markdown(officers, "Company Officers", selected_stock)
except Exception as e:
    display(Markdown(f"**Error displaying company officers:** {e}"))

In [ ]:
# Render Institutional Holdings as Markdown

# This cell is standalone — it only needs `ticker` to be defined (yf.Ticker object)
try:
    if 'ticker' not in globals():
        ticker = yf.Ticker(selected_stock)
    info = ticker.info or {}
    shares_outstanding = info.get('sharesOutstanding')

    institutional_holders = ticker.institutional_holders
    if institutional_holders is not None and not institutional_holders.empty:
        md_lines = []
        md_lines.append(f"### Top Institutional Holdings for {selected_stock}")
        md_lines.append('')

        held_pct = info.get('heldPercentInstitutions', None)
        if held_pct is not None:
            md_lines.append(f"**Percent held by Institutions:** {held_pct*100:.2f}%")
        else:
            md_lines.append("**Percent held by Institutions:** N/A")

        md_lines.append('')
        # Build numbered list of top 10 holders
        for i, row in enumerate(institutional_holders.head(10).itertuples(index=False), 1):
            # row typically has attributes 'Holder' and 'Shares' but structure may vary
            try:
                holder = getattr(row, 'Holder', None) or getattr(row, 'Holder', None) or list(row)[0]
            except Exception:
                holder = 'Unknown'
            try:
                shares = getattr(row, 'Shares', None) or getattr(row, 'Shares', None) or (list(row)[1] if len(list(row)) > 1 else 0)
            except Exception:
                shares = 0

            # Format shares
            shares_str = 'N/A'
            if shares and shares != 0:
                try:
                    shares_val = float(shares)
                    if shares_val >= 1e9:
                        shares_str = f"{shares_val/1e9:.1f}B shares"
                    elif shares_val >= 1e6:
                        shares_str = f"{shares_val/1e6:.1f}M shares"
                    else:
                        shares_str = f"{shares_val:,.0f} shares"

                    if shares_outstanding and shares_outstanding > 0:
                        pct = (shares_val / shares_outstanding) * 100
                        shares_str += f" ({pct:.2f}%)"
                except Exception:
                    shares_str = str(shares)

            md_lines.append(f"{i}. **{holder}** — {shares_str}")

        display(Markdown('\n\n'.join(md_lines)))
    else:
        display(Markdown(f"**Institutional Holdings:** Not available for {stock_name}"))
except Exception as e:
    display(Markdown(f"**Error fetching holdings data:** {e}"))

In [ ]:
# Render Mutual Fund Holdings as Markdown
from IPython.display import Markdown, display

# This cell is standalone — it only needs `ticker` to be defined (yf.Ticker object)
try:
    ticker  # ensure ticker exists
except NameError:
    ticker = yf.Ticker(selected_stock)

try:
    info = ticker.info or {}
    shares_outstanding = info.get('sharesOutstanding')

    mutualfund_holders = ticker.mutualfund_holders
    if mutualfund_holders is not None and not mutualfund_holders.empty:
        md_lines = []
        md_lines.append(f"### Top Mutual Fund Holdings for {selected_stock}")
        md_lines.append('')

        md_lines.append('')
        # Build numbered list of top 10 mutual fund holders
        for i, row in enumerate(mutualfund_holders.head(10).itertuples(index=False), 1):
            try:
                holder = getattr(row, 'Holder', None) or list(row)[0]
            except Exception:
                holder = 'Unknown'
            try:
                shares = getattr(row, 'Shares', None) or (list(row)[1] if len(list(row)) > 1 else 0)
            except Exception:
                shares = 0

            shares_str = 'N/A'
            if shares and shares != 0:
                try:
                    shares_val = float(shares)
                    if shares_val >= 1e9:
                        shares_str = f"{shares_val/1e9:.1f}B shares"
                    elif shares_val >= 1e6:
                        shares_str = f"{shares_val/1e6:.1f}M shares"
                    else:
                        shares_str = f"{shares_val:,.0f} shares"

                    if shares_outstanding and shares_outstanding > 0:
                        pct = (shares_val / shares_outstanding) * 100
                        shares_str += f" ({pct:.2f}%)"
                except Exception:
                    shares_str = str(shares)

            md_lines.append(f"{i}. **{holder}** — {shares_str}")

        display(Markdown('\n\n'.join(md_lines)))
    else:
        display(Markdown(f"**Mutual Fund Holdings:** Not available for {stock_name}"))
except Exception as e:
    display(Markdown(f"**Error fetching mutual fund holdings data:** {e}"))

In [ ]:
# Print shares outstanding for the selected stock
ticker = yf.Ticker(selected_stock)
info = ticker.info or {}
shares_out = info.get('sharesOutstanding')

def fmt_shares(n):
    try:
        n = float(n)
    except Exception:
        return str(n)
    if n >= 1e9:
        return f"{n/1e9:.2f}B shares"
    if n >= 1e6:
        return f"{n/1e6:.2f}M shares"
    return f"{n:,.0f} shares"

print(f"sharesOutstanding: {fmt_shares(shares_out)}")
# pprint(info)  # Uncomment if you need the full info dict


## Company Information

## Competitors
List of top five competitors for the selected stock

In [ ]:
# Fetch and display competitors from CNN Business API (self-contained, Markdown output)
from IPython.display import Markdown, display
import json

# requires `selected_stock` to be set (e.g. 'AAPL', 'GOOGL', etc)
api_url = f"https://production.dataviz.cnn.io/quote/competitors/{selected_stock}/10" #Can change the number at the end to get more competitors if available
desired_count = 10

competitors_list = []

# Make API request using utility function
resp = make_cnn_api_request(api_url)

md_lines = []
try:
    if resp is None:
        md_lines.append("**No response received (network error) — cannot fetch competitors.**")
    elif resp.status_code != 200:
        md_lines.append(f"**Final status:** {resp.status_code}. Response (truncated):")
        md_lines.append('')
        md_lines.append(resp.text[:1000])
    else:
        try:
            data = resp.json()
        except ValueError as e:
            md_lines.append("**Failed to decode JSON from response**")
            md_lines.append('')
            md_lines.append(resp.text[:2000])
            data = None

        if data is not None:
            # Normalize list/dict shapes
            if isinstance(data, dict) and "competitors" in data:
                items = data["competitors"]
            elif isinstance(data, list):
                items = data
            else:
                items = []

            def normalize_symbol(s):
                if not s:
                    return ""
                s = str(s).upper().strip()
                return s.split("-")[0].split(".")[0]

            target = normalize_symbol(selected_stock)

            if items:
                for comp in items:
                    sym = comp.get("symbol") or comp.get("ticker") or comp.get("ticker_symbol") or ""
                    sym_norm = normalize_symbol(sym)
                    if sym_norm == target:
                        continue
                    name = comp.get("name") or comp.get("companyName") or "Unknown"
                    competitors_list.append({"symbol": sym_norm or sym or "N/A", "name": name, "raw": comp})
                    if len(competitors_list) >= desired_count:
                        break

                if competitors_list:
                    md_lines.append(f"### Top {len(competitors_list)} competitors for {selected_stock}")
                    md_lines.append('')
                    for i, c in enumerate(competitors_list, 1):
                        md_lines.append(f"{i}. **{c['symbol']}** — {c['name']}")
                else:
                    md_lines.append("**No competitors found after excluding the stock itself.**")
                    md_lines.append('')
                    md_lines.append(json.dumps(items, indent=2)[:4000])
            else:
                md_lines.append("**No items in API response — dumping JSON (truncated):**")
                md_lines.append('')
                md_lines.append(json.dumps(data, indent=2)[:4000])

    display(Markdown('\n\n'.join(md_lines)))
except Exception as e:
    display(Markdown(f"**Error fetching competitors:** {e}"))

[Back to Top](#table-of-contents)

In [ ]:
# Quarterly and Yearly Income Statement and Balance Sheet with values in Billions

import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Quarterly data
revenue_q = ticker.quarterly_income_stmt.loc["Total Revenue"]
income_q = ticker.quarterly_income_stmt.loc["Net Income"]
assets_q = ticker.quarterly_balance_sheet.loc["Total Assets"]
debt_q = ticker.quarterly_balance_sheet.loc["Total Debt"]

# First subplot: Quarterly Total Revenue and Net Income with Profit Margin
revenue_b = revenue_q / 1e9
income_b = income_q / 1e9
df_plot1 = pd.DataFrame({'Revenue': revenue_b, 'Net Income': income_b})
df_plot1 = df_plot1.sort_index()  # Sort by date ascending
df_plot1 = df_plot1.tail(4)  # Show only the 4 most recent quarters
df_plot1.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=axes[0,0])
axes[0,0].set_title('Quarterly Total Revenue and Net Income (in Billions) with Profit Margin')
axes[0,0].set_ylabel('Amount (Billions USD)')
axes[0,0].set_xticks(range(len(df_plot1)))
axes[0,0].set_xticklabels([f"Q{((d.month-1)//3)+1} {d.year}" for d in df_plot1.index], rotation=0)
axes[0,0].grid(axis='y', linestyle='--', alpha=0.7)

# Calculate and plot Profit Margin on secondary axis for first subplot
profit_margin = (df_plot1['Net Income'] / df_plot1['Revenue']) * 100
ax1_twin = axes[0,0].twinx()
profit_margin_line_color = '#fb950f'  # Hex color for profit margin line
profit_margin_text_color = "#000000"  # Hex color for profit margin text/labels
ax1_twin.plot(range(len(df_plot1)), profit_margin.values, color=profit_margin_line_color, marker='o', linewidth=2, label='Profit Margin (%)')
ax1_twin.set_ylabel('Profit Margin (%)', color=profit_margin_text_color)
ax1_twin.tick_params(axis='y', labelcolor=profit_margin_text_color)
ax1_twin.set_ylim(bottom=0)  # Start profit margin axis at 0

# Combine legends for first subplot
lines1, labels1 = axes[0,0].get_legend_handles_labels()
lines1_twin, labels1_twin = ax1_twin.get_legend_handles_labels()
axes[0,0].legend(lines1 + lines1_twin, labels1 + labels1_twin, title='Metrics', loc='upper left')

# Second subplot: Quarterly Total Assets and Total Debt with Debt-to-Asset Ratio
assets_b = assets_q / 1e9
debt_b = debt_q / 1e9
df_plot2 = pd.DataFrame({'Total Assets': assets_b, 'Total Debt': debt_b})
df_plot2 = df_plot2.sort_index()  # Sort by date ascending
df_plot2 = df_plot2.tail(4)  # Show only the 4 most recent quarters
df_plot2.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=axes[0,1])
axes[0,1].set_title('Quarterly Total Assets and Total Debt (in Billions) with Debt-to-Asset Ratio')
axes[0,1].set_ylabel('Amount (Billions USD)')
axes[0,1].set_xticks(range(len(df_plot2)))
axes[0,1].set_xticklabels([f"Q{((d.month-1)//3)+1} {d.year}" for d in df_plot2.index], rotation=0)
axes[0,1].grid(axis='y', linestyle='--', alpha=0.7)

# Calculate and plot Debt-to-Asset Ratio on secondary axis for second subplot (Debt / Assets)
debt_to_asset_ratio = df_plot2['Total Debt'] / df_plot2['Total Assets']
debt_to_asset_ratio_pct = debt_to_asset_ratio * 100  # Convert to percentage
ax2_twin = axes[0,1].twinx()
ratio_line_color = '#fb950f'  # Red color for debt ratio
ratio_text_color = "#000000"  # Red color for text
ax2_twin.plot(range(len(df_plot2)), debt_to_asset_ratio_pct.values, color=ratio_line_color, marker='o', linewidth=3, linestyle='-', label='Debt-to-Asset Ratio (%)')
ax2_twin.set_ylabel('Debt-to-Asset Ratio (%)', color=ratio_text_color)
ax2_twin.tick_params(axis='y', labelcolor=ratio_text_color)
ax2_twin.set_ylim(bottom=0, top=100)  # Set to 0-100% range

# Combine legends for second subplot
lines2, labels2 = axes[0,1].get_legend_handles_labels()
lines2_twin, labels2_twin = ax2_twin.get_legend_handles_labels()
axes[0,1].legend(lines2 + lines2_twin, labels2 + labels2_twin, title='Metrics', loc='upper left')

# Yearly data
revenue_y = ticker.income_stmt.loc["Total Revenue"]
income_y = ticker.income_stmt.loc["Net Income"]
assets_y = ticker.balance_sheet.loc["Total Assets"]
debt_y = ticker.balance_sheet.loc["Total Debt"]

# Third subplot: Annual Total Revenue and Net Income with Profit Margin
revenue_b = revenue_y / 1e9
income_b = income_y / 1e9
df_plot3 = pd.DataFrame({'Revenue': revenue_b, 'Net Income': income_b})
df_plot3 = df_plot3.sort_index()  # Sort by date ascending
df_plot3 = df_plot3.tail(4)  # Show only the 4 most recent years
df_plot3.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=axes[1,0])
axes[1,0].set_title('Annual Total Revenue and Net Income (in Billions) with Profit Margin')
axes[1,0].set_ylabel('Amount (Billions USD)')
axes[1,0].set_xticks(range(len(df_plot3)))
axes[1,0].set_xticklabels([d.year for d in df_plot3.index], rotation=0)
axes[1,0].grid(axis='y', linestyle='--', alpha=0.7)

# Calculate and plot Profit Margin on secondary axis for third subplot
profit_margin = (df_plot3['Net Income'] / df_plot3['Revenue']) * 100
ax3_twin = axes[1,0].twinx()
ax3_twin.plot(range(len(df_plot3)), profit_margin.values, color=profit_margin_line_color, marker='o', linewidth=2, label='Profit Margin (%)')
ax3_twin.set_ylabel('Profit Margin (%)', color=profit_margin_text_color)
ax3_twin.tick_params(axis='y', labelcolor=profit_margin_text_color)
ax3_twin.set_ylim(bottom=0)  # Start profit margin axis at 0

# Combine legends for third subplot
lines3, labels3 = axes[1,0].get_legend_handles_labels()
lines3_twin, labels3_twin = ax3_twin.get_legend_handles_labels()
axes[1,0].legend(lines3 + lines3_twin, labels3 + labels3_twin, title='Metrics', loc='upper left')

# Fourth subplot: Annual Total Assets and Total Debt with Debt-to-Asset Ratio
assets_b = assets_y / 1e9
debt_b = debt_y / 1e9
df_plot4 = pd.DataFrame({'Total Assets': assets_b, 'Total Debt': debt_b})
df_plot4 = df_plot4.sort_index()  # Sort by date ascending
df_plot4 = df_plot4.tail(4)  # Show only the 4 most recent years
df_plot4.plot(kind='bar', color=["#000000", "#2a5fff"], width=0.8, ax=axes[1,1])
axes[1,1].set_title('Annual Total Assets and Total Debt (in Billions) with Debt-to-Asset Ratio')
axes[1,1].set_ylabel('Amount (Billions USD)')
axes[1,1].set_xticks(range(len(df_plot4)))
axes[1,1].set_xticklabels([d.year for d in df_plot4.index], rotation=0)
axes[1,1].grid(axis='y', linestyle='--', alpha=0.7)

# Calculate and plot Debt-to-Asset Ratio on secondary axis for fourth subplot (Debt / Assets)
debt_to_asset_ratio = df_plot4['Total Debt'] / df_plot4['Total Assets']
debt_to_asset_ratio_pct = debt_to_asset_ratio * 100  # Convert to percentage
ax4_twin = axes[1,1].twinx()
ax4_twin.plot(range(len(df_plot4)), debt_to_asset_ratio_pct.values, color=ratio_line_color, marker='o', linewidth=3, linestyle='-', label='Debt-to-Asset Ratio (%)')
ax4_twin.set_ylabel('Debt-to-Asset Ratio (%)', color=ratio_text_color)
ax4_twin.tick_params(axis='y', labelcolor=ratio_text_color)
ax4_twin.set_ylim(bottom=0, top=100)  # Set to 0-100% range

# Combine legends for fourth subplot
lines4, labels4 = axes[1,1].get_legend_handles_labels()
lines4_twin, labels4_twin = ax4_twin.get_legend_handles_labels()
axes[1,1].legend(lines4 + lines4_twin, labels4 + labels4_twin, title='Metrics', loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Display annual balance sheet with all rows visible
import pandas as pd
bs = ticker.balance_sheet
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(bs)

In [ ]:
ticker.insider_roster_holders

In [ ]:
ticker.insider_transactions.head(10)

In [ ]:
# Insider Transactions
# Choose symbol: use selected_stock if available, otherwise prompt
try:
    sym = selected_stock
except NameError:
    sym = input('Enter ticker symbol: ').strip().upper()

ticker = yf.Ticker(sym)
ins = None
try:
    ins = ticker.insider_transactions
except Exception as e:
    print('Could not fetch insider_transactions:', e)

if ins is None or ins.empty:
    print('No insider transactions available for', sym)
else:
    df_ins = ins.copy()

    # Helper to format large numbers
    def humanize_number(x):
        try:
            x = float(x)
        except Exception:
            return '?'
        if pd.isna(x):
            return '?'
        abs_x = abs(x)
        if abs_x >= 1e9:
            return f'{x/1e9:.2f}B'
        if abs_x >= 1e6:
            return f'{x/1e6:.2f}M'
        if abs_x >= 1e3:
            return f'{x/1e3:.2f}K'
        return f'{x:.0f}'

    # Sort by date if available
    date_col = next((c for c in ['Start Date', 'FilingDate', 'Date', 'TransactionDate'] if c in df_ins.columns), None)
    if date_col:
        df_ins.sort_values(by=date_col, ascending=False, inplace=True)

    # Top 10 newest
    df_top = df_ins.head(10)

    markdown_lines = [f"### Insider Transactions for {selected_stock}\n"]

    for _, row in df_top.iterrows():
        insider = row.get('Insider', 'Unknown')
        title = row.get('Title', row.get('Position', '')).strip()
        text = row.get('Text', '')  # descriptive text
        shares = row.get('Shares', row.get('SharesTransacted', row.get('SharesChanged', None)))
        value = row.get('Value', row.get('ValueTransacted', None))

        # Format values
        shares_h = humanize_number(shares)
        value_h = humanize_number(value)
        date = row.get(date_col, '')
        date_str = f"{pd.to_datetime(date).date()}" if pd.notna(date) else 'Unknown date'

        # Combine title and insider name
        who = f"**{title}**, {insider}" if title else f"{insider}"

        # Construct Markdown line
        if text:
            action_desc = text.strip()
        else:
            action_desc = "Transaction"

        markdown_lines.append(
            f"- {date_str}: {who} **{action_desc}** ({shares_h} shares) for an overall value of **${value_h}**"
        )

    # Display as Markdown
    display(Markdown("\n".join(markdown_lines)))

In [ ]:
# Render latest news articles from yfinance as Markdown (self-contained)
from IPython.display import Markdown, display

# Render ticker.news as wrapped Markdown (one article per numbered item)
try:
    news = getattr(ticker, 'news', None)
    md_lines = []

    if not news:
        md_lines.append("**No news available for this stock.**")
    else:
        md_lines.append(f"### News for {selected_stock} — {len(news)} articles found")
        md_lines.append('')

        for i, article in enumerate(news, 1):
            try:
                # Extract content (news data is nested under 'content' key)
                content = article.get('content', {}) if isinstance(article, dict) else {}

                # Extract provider from nested structure
                provider_obj = content.get('provider', {}) if isinstance(content, dict) else {}
                provider = provider_obj.get('displayName', 'Unknown') if isinstance(provider_obj, dict) else 'Unknown'

                # Extract summary
                summary = content.get('summary') or content.get('description') or content.get('title') or 'No summary available'

                # Extract pubDate
                pub_date = content.get('pubDate') or content.get('displayTime') or article.get('pubDate') or article.get('date') or 'Unknown date'

                # Format date if it's in ISO format
                if pub_date and pub_date != 'Unknown date':
                    try:
                        from datetime import datetime
                        dt = datetime.fromisoformat(str(pub_date).replace('Z', '+00:00'))
                        pub_date = dt.strftime('%Y-%m-%d %H:%M:%S')
                    except Exception:
                        pub_date = str(pub_date)

                # Truncate summary to 150 characters for one-line display
                summary_short = summary[:150] + '...' if isinstance(summary, str) and len(summary) > 150 else summary

                # Build Markdown line: numbered item with provider, bolded, summary, and date in parentheses
                md_lines.append(f"{i}. **{provider}** — {summary_short} ({pub_date})")
            except Exception as e:
                md_lines.append(f"{i}. Error parsing article: {e}")

    display(Markdown('\n\n'.join(md_lines)))
except Exception as e:
    display(Markdown(f"**Error rendering news:** {e}"))

In [ ]:
# Take a closer look at this - Analyst upgrades and downgrades data
ticker.upgrades_downgrades.head(10)

# Enhancement Idea: Improve Analyst Price Targets Visualization

Create a chart that shows the stock price history in the first figure and extends it to the right with a distribution of current price targets from analyst upgrades/downgrades data.

## Analyst Price Targets

## Analyst Price Targets Visualization

This chart combines the stock price history with analyst price target distributions from upgrades/downgrades data.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np

def get_filtered_targets(ticker):
    """Get filtered analyst price targets from ticker recommendations or info."""
    targets = []
    
    # Try to get from recommendations
    try:
        recs = ticker.recommendations
        if recs is not None and 'priceTarget' in recs.columns:
            targets = recs['priceTarget'].dropna().astype(float).tolist()
    except:
        pass
    
    # If no targets from recommendations, try info
    if not targets:
        try:
            info = ticker.info
            mean_target = info.get('targetMeanPrice')
            high_target = info.get('targetHighPrice')
            low_target = info.get('targetLowPrice')
            if mean_target:
                targets.append(mean_target)
            if high_target and high_target != mean_target:
                targets.append(high_target)
            if low_target and low_target != mean_target:
                targets.append(low_target)
        except:
            pass
    
    # Filter unreasonable targets
    if targets:
        current_price = ticker.info.get('currentPrice', ticker.info.get('regularMarketPrice', 0))
        if current_price > 0:
            # Keep targets within 50% to 200% of current price
            filtered = [t for t in targets if 0.5 * current_price <= t <= 2 * current_price]
            return filtered
    return targets

def plot_price_chart(ax, data, stock_symbol, price_targets):
    """Plot the main price chart with targets."""
    if 'Close' not in data.columns:
        raise ValueError("Data must contain 'Close' column")
    
    # Plot price
    ax.plot(data.index, data['Close'], label='Close Price', color='blue', linewidth=2)
    
    # Plot targets as horizontal lines
    if price_targets:
        avg_target = np.mean(price_targets)
        ax.axhline(y=avg_target, color='green', linestyle='--', label=f'Avg Target: ${avg_target:.2f}')
        for target in price_targets:
            ax.axhline(y=target, color='red', alpha=0.3, linewidth=1)
    
    ax.set_title(f'{stock_symbol} Price Chart with Analyst Targets')
    ax.set_ylabel('Price ($)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    return data['Close'].iloc[-1]

def calculate_price_range(close_prices, price_targets):
    """Calculate the price range for plotting."""
    min_close = close_prices.min()
    max_close = close_prices.max()
    
    if price_targets:
        min_target = min(price_targets)
        max_target = max(price_targets)
        price_min = min(min_close, min_target)
        price_max = max(max_close, max_target)
    else:
        price_min = min_close
        price_max = max_close
    
    # Add some padding
    padding = (price_max - price_min) * 0.1
    return price_min - padding, price_max + padding

def plot_target_distribution(ax, price_targets, current_price, price_min, price_max):
    """Plot the distribution of analyst price targets as horizontal lines."""
    if not price_targets:
        ax.text(0.5, 0.5, 'No analyst targets available', 
                transform=ax.transAxes, ha='center', va='center')
        ax.set_xlim(0, 1)
        ax.set_ylim(price_min, price_max)
        return
    
    # Plot each target as a horizontal line
    for i, target in enumerate(price_targets):
        color = 'green' if target > current_price else 'red' if target < current_price else 'blue'
        ax.axhline(y=target, color=color, linewidth=2, alpha=0.8, label=f'Target {i+1}: ${target:.2f}')
    
    # Plot current price
    ax.axhline(y=current_price, color='blue', linewidth=3, linestyle='--', label=f'Current: ${current_price:.2f}')
    
    ax.set_xlabel('Targets')
    ax.set_title('Analyst Targets')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 1)  # Fixed x-axis since we're just showing lines

def print_analyst_summary(stock_symbol, current_price, price_targets):
    """Print analyst summary."""
    print(f"\n=== Analyst Summary for {stock_symbol} ===")
    print(f"Current Price: ${current_price:.2f}")
    
    if not price_targets:
        print("No analyst price targets available.")
        return
    
    avg_target = np.mean(price_targets)
    min_target = min(price_targets)
    max_target = max(price_targets)
    
    print(f"Average Target: ${avg_target:.2f}")
    print(f"Target Range: ${min_target:.2f} - ${max_target:.2f}")
    print(f"Number of Targets: {len(price_targets)}")
    
    upside = ((avg_target - current_price) / current_price) * 100
    print(f"Upside Potential: {upside:.1f}%")
    
    # Count targets above/below current
    above = sum(1 for t in price_targets if t > current_price)
    below = sum(1 for t in price_targets if t < current_price)
    print(f"Targets Above Current: {above}")
    print(f"Targets Below Current: {below}")

In [ ]:
# Main execution
try:
    # Get data
    price_targets = get_filtered_targets(ticker)
    data = stock_metrics[selected_stock]

    # Create figure
    fig = plt.figure(figsize=(18, 8))
    gs = GridSpec(1, 20, wspace=0.05)

    # Main price chart
    ax_main = fig.add_subplot(gs[0, :16])
    current_price = plot_price_chart(ax_main, data, selected_stock, price_targets)

    # Target distribution
    ax_dist = fig.add_subplot(gs[0, 16:], sharey=ax_main)
    ax_dist.yaxis.tick_right()
    ax_dist.yaxis.set_label_position('right')
    ax_dist.yaxis.set_ticks_position('right')
    ax_dist.tick_params(labelleft=False, labelright=True)

    # Calculate price range and plot distribution
    price_min, price_max = calculate_price_range(data['Close'], price_targets)
    plot_target_distribution(ax_dist, price_targets, current_price, price_min, price_max)

    # Final styling
    ax_main.tick_params(axis='y', which='both', labelleft=False, labelright=True)
    plt.tight_layout()
    plt.show()

    # Print summary
    print_analyst_summary(selected_stock, current_price, price_targets)

except Exception as e:
    print(f"Error creating analyst target chart: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Plot Recommendations Summary as a column (bar) chart

# Ensure `ticker` and `selected_stock` exist (run stock selection cell first)
try:
    rec = ticker.recommendations_summary
except (NameError, AttributeError):
    display(Markdown('**`ticker` is not defined or has no recommendations_summary. Run the STOCK SELECTION cell first.**'))
    rec = None

if rec is None or (isinstance(rec, pd.DataFrame) and rec.empty):
    display(Markdown('**No recommendations_summary available for this ticker.**'))
else:
    try:
        # recommendations_summary is typically a DataFrame with columns like:
        # period, strongBuy, buy, hold, sell, strongSell
        # We need to use ONLY THE MOST RECENT row (latest period)
        
        if isinstance(rec, pd.DataFrame):
            # Get the most recent row (last row in the DataFrame)
            latest_row = rec.iloc[-1]
            
            recommendation_counts = {}
            
            # Check various column name formats
            col_mapping = {
                'strongBuy': 'Strong Buy',
                'strong_buy': 'Strong Buy',
                'Strong Buy': 'Strong Buy',
                'buy': 'Buy',
                'Buy': 'Buy',
                'hold': 'Hold',
                'Hold': 'Hold',
                'sell': 'Sell',
                'Sell': 'Sell',
                'strongSell': 'Strong Sell',
                'strong_sell': 'Strong Sell',
                'Strong Sell': 'Strong Sell'
            }
            
            for col in rec.columns:
                if col in col_mapping:
                    label = col_mapping[col]
                    # Get value from the latest row only
                    value = latest_row[col]
                    if pd.notna(value) and value > 0:
                        recommendation_counts[label] = value
            
            if not recommendation_counts:
                display(Markdown('**No valid recommendation data found in the most recent period.**'))
            else:
                # Create Series from the counts
                s = pd.Series(recommendation_counts)
                
                # Order labels in the preferred order
                preferred_order = ['Strong Buy', 'Buy', 'Hold', 'Sell', 'Strong Sell']
                present = [p for p in preferred_order if p in s.index]
                s = s.reindex(present)
                
                # Plot vertical column chart
                fig, ax = plt.subplots(figsize=(10, 6))
                
                # Define colors for each category
                colors = []
                for label in s.index:
                    if 'Strong Buy' in label:
                        colors.append('#00aa00')  # Dark green
                    elif 'Buy' in label:
                        colors.append('#66cc66')  # Light green
                    elif 'Hold' in label:
                        colors.append('#ffaa00')  # Orange
                    elif 'Sell' in label and 'Strong' not in label:
                        colors.append('#ff6666')  # Light red
                    elif 'Strong Sell' in label:
                        colors.append('#cc0000')  # Dark red
                    else:
                        colors.append('#7f7f7f')  # Gray
                
                bars = ax.bar(s.index, s.values, color=colors, edgecolor='black', alpha=0.85, width=0.6)
                
                # Get the period/date if available
                period_info = ""
                if 'period' in rec.columns:
                    period_info = f" (Period: {latest_row['period']})"
                
                ax.set_title(f'Current Analyst Recommendations — {selected_stock}{period_info}', fontsize=14, fontweight='bold')
                ax.set_ylabel('Number of Analysts', fontsize=12)
                ax.set_xlabel('Recommendation', fontsize=12)
                plt.xticks(rotation=45, ha='right')
                ax.grid(axis='y', alpha=0.3, linestyle='--')
                
                # Annotate values on top of bars
                for bar in bars:
                    h = bar.get_height()
                    if h > 0:
                        ax.annotate(f'{int(h)}', 
                                  xy=(bar.get_x() + bar.get_width() / 2, h), 
                                  xytext=(0, 3),
                                  textcoords='offset points', 
                                  ha='center', 
                                  va='bottom', 
                                  fontsize=11,
                                  fontweight='bold')
                
                plt.tight_layout()
                plt.show()
        else:
            display(Markdown(f'**Unexpected data type for recommendations_summary: {type(rec)}**'))
            
    except Exception as e:
        display(Markdown(f'**Error preparing recommendations plot:** {e}'))
        import traceback
        print(traceback.format_exc())

[Back to Top](#table-of-contents)

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from datetime import datetime, timezone

import pandas as pd
from IPython.display import display

url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"

headers_list = [
    {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Referer": "https://www.cnn.com/",
    },
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Referer": "https://www.cnn.com/",
    },
    {"User-Agent": "curl/7.88.1", "Accept": "*/*"},
]

session = requests.Session()
retries = Retry(total=2, backoff_factor=0.8, status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

resp = None
for h in headers_list:
    try:
        resp = session.get(url, headers=h, timeout=8)
        if resp.status_code == 200:
            break
    except requests.exceptions.RequestException:
        resp = None

if resp is None:
    print("No response received (network error)")
else:
    data = resp.json()

    metric_keys = [
        'fear_and_greed', 'fear_and_greed_historical', 'market_momentum_sp500',
        'market_momentum_sp125', 'stock_price_strength', 'stock_price_breadth',
        'put_call_options', 'market_volatility_vix', 'market_volatility_vix_50',
        'junk_bond_demand', 'safe_haven_demand'
    ]

    def parse_ts(ts):
        if ts is None:
            return pd.NaT
        try:
            if isinstance(ts, (int, float)):
                t = float(ts)
                if t > 1e12:
                    return pd.to_datetime(t, unit='ms', utc=True)
                elif t > 1e9:
                    return pd.to_datetime(t, unit='s', utc=True)
            if isinstance(ts, str):
                try:
                    return pd.to_datetime(ts)
                except Exception:
                    return pd.NaT
        except Exception:
            return pd.NaT
        return pd.NaT

    rows = []
    for mk in metric_keys:
        entry = data.get(mk) if isinstance(data, dict) else None
        score = None
        rating = None
        timestamp = None

        if isinstance(entry, dict):
            score = entry.get('score')
            rating = entry.get('rating')
            timestamp = entry.get('timestamp')
            if (score is None or rating is None or timestamp is None) and isinstance(entry.get('data'), list) and entry.get('data'):
                last = entry['data'][-1]
                if isinstance(last, dict):
                    score = score if score is not None else (last.get('y') or last.get('value'))
                    rating = rating if rating is not None else (last.get('rating') or last.get('label'))
                    timestamp = timestamp if timestamp is not None else last.get('x')
        rows.append({'metric': mk, 'score': score, 'rating': rating, 'timestamp': parse_ts(timestamp)})

    df = pd.DataFrame(rows).set_index('metric')
    # Try to convert score to numeric
    df['score'] = pd.to_numeric(df['score'], errors='coerce')

    display(df)

# Enhancement Idea: Create Multiple Charts Showing Development Over Time

Create X number of graphs that show the development of key metrics over time for better trend analysis.

## Market Intelligence

In [ ]:
# Fetch share-price insights and display only list_summary.plain_text (if present)
from IPython.display import Markdown, display
import json

# requires `selected_stock` to be set (e.g. 'AAPL', 'GOOGL', etc)
api_url = f"https://production.dataviz.cnn.io/insights/share_price/{selected_stock}"

# Make API request using utility function
resp = make_cnn_api_request(api_url)

try:
    if resp is None:
        display(Markdown("**No response received (network error) — cannot fetch insights.**"))
    elif resp.status_code != 200:
        # Show truncated response body for debugging
        display(Markdown(f"**Final status:** {resp.status_code}. Response (truncated):\n\n{resp.text[:1000]}"))
    else:
        try:
            data = resp.json()
        except ValueError:
            display(Markdown("**Failed to decode JSON from response**"))
            display(Markdown(resp.text[:2000]))
            data = None

        # Try to extract list_summary -> plain_text with a few common key-name fallbacks
        def extract_plain_text(obj):
            # Direct dict with expected key
            if not isinstance(obj, (dict, list)):
                return None
            # Common top-level keys
            candidates = ['list_summary', 'listSummary', 'listsummary']
            for key in candidates:
                if isinstance(obj, dict) and key in obj:
                    val = obj[key]
                    if isinstance(val, dict):
                        for pk in ('plain_text', 'plainText', 'plaintext'):
                            if pk in val and isinstance(val[pk], str):
                                return val[pk].strip()
                    if isinstance(val, str):
                        return val.strip()
            # Check common nested places (e.g., 'summary', 'data' lists, etc.)
            if isinstance(obj, dict):
                for v in obj.values():
                    res = extract_plain_text(v)
                    if res:
                        return res
            if isinstance(obj, list):
                for item in obj:
                    res = extract_plain_text(item)
                    if res:
                        return res
            return None

        if data is not None:
            plain = extract_plain_text(data)
            if plain:
                # Display only the extracted plaintext
                display(Markdown(plain))
            else:
                # Fallback: show small dump to help debugging
                display(Markdown("**No list_summary.plain_text found — dumping top-level JSON (truncated):**"))
                display(Markdown(json.dumps(data, indent=2)[:4000]))
except Exception as e:
    display(Markdown(f"**Error fetching insights:** {e}"))

In [ ]:
# Real-time stock price streaming via WebSocket (BTC-USD example)
#!pip install yfinance nest_asyncio

# Import modules
import nest_asyncio
import asyncio
import yfinance as yf

# Allow nested async loops (needed for Jupyter)
nest_asyncio.apply()

# Define a message handler
def message_handler(message):
    # Each message is a dict with price updates
    if "id" in message and message["id"] == "BTC-USD":
        price = message.get("price")
        if price:
            print(f"BTC-USD: ${price:.2f}")

# Create async function for WebSocket 
async def stream_btc():
    async with yf.AsyncWebSocket(verbose=False) as ws:
        await ws.subscribe(["BTC-USD"])
        print("Connected to Yahoo Finance WebSocket. Streaming BTC-USD live...")
        await ws.listen(message_handler)

# Run it
#await stream_btc()         # Uncomment this line to start streaming (will run indefinitely)

In [ ]:
import pandas as pd
import yfinance as yf
import folium
import requests
from datetime import datetime, timedelta

# Step 1: Define major stock indices with full country names (matching GeoJSON)
country_indices = {
    'United States of America': '^GSPC',      # S&P 500
    'United Kingdom': '^FTSE',                # FTSE 100
    'Germany': '^GDAXI',                      # DAX
    'France': '^FCHI',                        # CAC 40
    'Japan': '^N225',                         # Nikkei 225
    'China': '000001.SS',                     # Shanghai Composite
    'India': '^BSESN',                        # BSE Sensex
    'Brazil': '^BVSP',                        # Bovespa
    'Canada': '^GSPTSE',                      # S&P/TSX
    'Australia': '^AXJO',                     # ASX 200
    'South Korea': '^KS11',                   # KOSPI
    'Mexico': '^MXX',                         # IPC Mexico
    'South Africa': '^JN0U.JO',               # JSE All Share
    'Spain': '^IBEX',                         # IBEX 35
    'Italy': 'FTSEMIB.MI',                    # FTSE MIB
    'Netherlands': '^AEX',                    # AEX
    'Switzerland': '^SSMI',                   # SMI
    'Sweden': '^OMX',                         # OMX Stockholm
    'Hong Kong': '^HSI',                      # Hang Seng
    'Singapore': '^STI',                      # Straits Times
    'Taiwan': '^TWII',                        # Taiwan Weighted
    'Indonesia': '^JKSE',                     # Jakarta Composite
    'Malaysia': '^KLSE',                      # FTSE Bursa Malaysia KLCI
    'Thailand': '^SET.BK',                    # SET Index
    'Philippines': '^PSI.PS',                 # PSE Composite
    'Norway': '^OSEAX',                       # Oslo Børs All-Share
    'Denmark': '^OMXC25',                     # OMX Copenhagen 25
    'Finland': '^OMXH25',                     # OMX Helsinki 25
    'Belgium': '^BFX',                        # BEL 20
    'Austria': '^ATX',                        # ATX
    'Portugal': '^PSI20',                     # PSI 20
    'Greece': 'ASEGR.AT',                     # Athens General Composite
    'Turkey': 'XU100.IS',                     # BIST 100
    'Poland': 'WIG20.WA',                     # WIG20
    'Russia': 'IMOEX.ME',                     # MOEX Russia
    'Israel': '^TA125.TA',                    # TA-125
    'Saudi Arabia': 'TASI.SR',               # Tadawul All Share
    'United Arab Emirates': 'ADI.AE',        # ADX General
    'Egypt': 'EGX30.CA',                      # EGX 30
    'Argentina': '^MERV',                     # MERVAL
    'Chile': '^IPSA',                         # S&P/CLX IPSA
    'Colombia': '^COLCAP',                    # COLCAP
    'Peru': '^SPBLPGPT',                      # S&P/BVL Peru General
    'New Zealand': '^NZ50',                   # S&P/NZX 50
    'Pakistan': '^KSE100',                    # KSE 100
    'Bangladesh': 'DSEX.DH',                  # DSEX
    'Vietnam': '^VNINDEX',                    # VN-Index
}

# Fetch stock data and calculate daily change
def get_daily_changes():
    import warnings
    import sys
    from contextlib import redirect_stderr
    import io
    
    # Suppress yfinance warnings
    warnings.filterwarnings('ignore', category=FutureWarning, module='yfinance')
    
    # Capture stderr to suppress yfinance error messages
    stderr_capture = io.StringIO()
    
    data = []
    with redirect_stderr(stderr_capture):
        for country, ticker in country_indices.items():
            try:
                # Get last 2 days of data
                stock = yf.Ticker(ticker)
                hist = stock.history(period='5d')
                
                if len(hist) >= 2:
                    # Calculate daily change percentage
                    latest_close = hist['Close'].iloc[-1]
                    previous_close = hist['Close'].iloc[-2]
                    daily_change = ((latest_close - previous_close) / previous_close) * 100
                    
                    data.append({
                        'country': country,
                        'ticker': ticker,
                        'daily_change': daily_change,
                        'close': latest_close
                    })
                    # print(f"{country}: {daily_change:.2f}%")
            except Exception as e:
                # Skip tickers that can't be found without printing errors
                pass
    
    return pd.DataFrame(data)

# Get the data
df = get_daily_changes()

# Create the Folium map with choropleth
def create_world_map(df):
    # Create base map (no zoom controls)
    m = folium.Map(
        location=[20, 0],
        zoom_start=2,
        tiles='CartoDB positron',
        zoom_control=False,
        scrollWheelZoom=False,
        doubleClickZoom=False,
        boxZoom=False,
        keyboard=False
    )
    
    # Get world GeoJSON
    world_geo = requests.get(
        'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
    ).json()
    
    # Create custom color function to match marker colors
    def get_country_color(daily_change):
        if daily_change >= 0:
            return '#008000'  # Green for gains
        else:
            return '#FF0000'  # Red for losses
    
    # Create choropleth layer with custom colors
    folium.Choropleth(
        geo_data=world_geo,
        name='Stock Index Daily Change',
        data=df,
        columns=['country', 'daily_change'],
        key_on='feature.properties.name',  # Match by country name
        fill_color='RdYlGn',  # Keep the gradient legend
        fill_opacity=0.8,  # Slightly more opaque
        line_opacity=0.3,
        legend_name='Daily Change (%)',
        nan_fill_color='lightgray',
        nan_fill_opacity=0.4,
    ).add_to(m)
    
    # Override the choropleth colors with our custom gradient scheme
    # We'll add a GeoJson layer on top with the correct colors
    def style_function(feature):
        country_name = feature['properties']['name']
        matching_row = df[df['country'] == country_name]
        if not matching_row.empty:
            daily_change = matching_row['daily_change'].iloc[0]
            
            # Create gradient color mapping
            if daily_change <= -2:
                color = '#8B0000'  # Dark red for strong losses
            elif daily_change <= -1:
                color = '#FF0000'  # Red for moderate losses
            elif daily_change < 0:
                color = '#FF8C00'  # Orange for small losses
            elif daily_change <= 1:
                color = '#FFFF00'  # Yellow for small gains
            elif daily_change <= 2:
                color = '#90EE90'  # Light green for moderate gains
            else:
                color = '#006400'  # Dark green for strong gains
                
            return {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.8
            }
        return {
            'fillColor': 'lightgray',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.4
        }
    
    # Add individual GeoJson layers for each country with popups
    for idx, row in df.iterrows():
        # Find the country in GeoJSON
        for feature in world_geo['features']:
            if feature['properties']['name'] == row['country']:
                # Create popup text
                popup_text = f"""
                <div style='font-family: Arial; width: 200px;'>
                    <b style='font-size: 14px;'>{row['country']}</b><br>
                    <hr style='margin: 5px 0;'>
                    <b>Index:</b> {row['ticker']}<br>
                    <b>Daily Change:</b> <span style='color: {"green" if row["daily_change"] >= 0 else "red"}; font-size: 16px; font-weight: bold;'>{row['daily_change']:+.2f}%</span><br>
                    <b>Close:</b> {row['close']:.2f}
                </div>
                """

                # Get country color
                if row['daily_change'] <= -2:
                    color = '#8B0000'  # Dark red
                elif row['daily_change'] <= -1:
                    color = '#FF0000'  # Red
                elif row['daily_change'] < 0:
                    color = '#FF8C00'  # Orange
                elif row['daily_change'] <= 1:
                    color = '#FFFF00'  # Yellow
                elif row['daily_change'] <= 2:
                    color = '#90EE90'  # Light green
                else:
                    color = '#006400'  # Dark green

                # Create individual GeoJson for this country
                country_geo = {
                    'type': 'FeatureCollection',
                    'features': [feature]
                }

                folium.GeoJson(
                    country_geo,
                    style_function=lambda x, color=color: {
                        'fillColor': color,
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.8
                    },
                    popup=folium.Popup(popup_text, max_width=250),
                    name=row['country']
                ).add_to(m)
                break
    
    folium.LayerControl().add_to(m)
    return m

# Create and save the map
world_map = create_world_map(df)
world_map.save('stock_index_world_map.html')
# print("\nMap saved as 'stock_index_world_map.html'")

# Display in Jupyter
world_map

[Back to Top](#table-of-contents)